# OTI MONITORING SET UP - STEP 2 - CENTRALIZING DATA IN A DIFFERENT REGION  
 - SOURCE ACCOUNT IN A DIFFERENT REGION FROM THE TARGET ACCOUNT

 METHOD: REPLICATION
 
Because we can't share across regions without replicating to an account in that region.  In this case we're replicating it directly to the target account. 

## Enable Replication for Source Accounts in a Different Region
**In an account with the orgadmin role enabled**, eneable replication for each source account that is in a different region than the  target account. 

In [ ]:
--In an ORG ADMIN enabled account - 
-- Assume the ORGADMIN role
USE ROLE ORGADMIN;
-- View the list of the accounts
SHOW ORGANIZATION ACCOUNTS;
-- Enable replication for each account that is in a different region than the OTI Organization account (TARGET ACCOUNT) that you will use form monitoring the accounts
SELECT SYSTEM$GLOBAL_ACCOUNT_SET_PARAMETER('<account1_locator>', 'ENABLE_ACCOUNT_DATABASE_REPLICATION', 'true');

### IN THE SOURCE ACCOUNT, CREATION A REPLICATION GROUP

Create a replication group for replicating the objects to the Target Account 

In the script below
- Supply an replication group name on line 4
    - Must be unique accross all accounts, recommend including the current account name
- Supply the target organization on line 8
- Change the schedule, if desired, on line 9

In [ ]:
--***********  if account is in a different region than the TARGET ACCOUNT -- CREATE REPLICATION GROUP ****************************
USE ROLE ACCOUNT_MONITOR_OTI;
set obs = 'MONITORING_SHARE_' || current_account_name();
-- Create REPLICATION Group
CREATE REPLICATION GROUP <replication_grou_name>  --unique, recommend including current_account
IGNORE EDITION CHECK
OBJECT_TYPES = DATABASES 
ALLOWED_DATABASES = IDENTIFIER($obs)
ALLOWED_ACCOUNTS = <ORGANIZATION.TARGET_ACCOUNT_NAME>; --CHANGE THIS TO THE TARGET ACCOUNT
REPLICATION_SCHEDULE = '1440 MINUTE';

In [ ]:
--GET SHARE NAME 
SELECT $obs;

### IN THE TARGET ACCOUNT, CREATION A REPLICA OF THE REPLICATION GROUP CREATED ABOVE

Create a replication group for replicating the objects to the Target Account 

In the script below:
- Supply the replication group name on line 7
    - Must be unique accross all accounts, recommend including the current account name
- Supply the organization_name, source_account, AND replication_group_name on line 9


In [ ]:
---IN TARGET ACCOUNT
USE ROLE ACCOUNTADMIN;
GRANT CREATE REPLICATION GROUP ON ACCOUNT TO ROLE ACCOUNT_MONITOR;

USE ROLE ACCOUNT_MONITOR;
-- Create Secondary Failover Group
CREATE FAILOVER GROUP <replication_group_name>  --USE THE RESULT OF THE SELECT $obs   STATEMENT RUN IN THE SOURCE ACCOUNT;
AS REPLICA OF <organization_name>.<source_account>.<replication_group_name>;

### IN THE TARGET ACCOUNT, MANUALLY TRIGGER REPLICATION USING REFRESH

Trigger the inital replication using refresh 

In the script below:
- Supply the replication group name on line 2


In [ ]:
-- Run initial replication manually
ALTER REPLICATION GROUP <replication_group_name> REFRESH;




### IN THE TARGET ACCOUNT, VALIDATE THE REFRESH 

Validate the refresh by selecting from the TRUST_CENTER.FINDINGS table

In the script below:
- Supply the database name on line 2

In [ ]:
--TEST SELECTING FROM SHARED OBJECTS
select * from <DATABASE_NAME>.TRUST_CENTER.FINDINGS;
--***END -- IN THE TARGET ACCOUNT